In [1]:
#Global Stuff, usually this can just be ran without issues
import base64
import io
from pprint import pprint
from functions import *

import requests
from PIL import Image, PngImagePlugin

url = "http://127.0.0.1:7860"  # Might need to change to your Stablediffusion UI Hostname/IP


In [2]:
def initialise(seed, prompt, negative_prompt, steps, samplers=None):
    """
    Function generates a default payload, and sets default sampler list if none is provided
    """
    if seed == -1:
        import random
        seed = random.randint(1000000000, 10000000000)
    payload = {
        "enable_hr": False,
        #"denoising_strength": 0,
        #"firstphase_width": 0,
        #"firstphase_height": 0,
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        #"styles": [
        #  "Good Baseline NSFW",
        #  "Good Baseline NSFW"
        #],
        "seed": seed,
        #"subseed": -1,
        #"subseed_strength": 0,
        #"seed_resize_from_h": -1,
        #"seed_resize_from_w": -1,
        #"sampler_name": "string",
        "batch_size": 1,
        "n_iter": 1,
        "steps": steps,
        "cfg_scale": 7,
        "width": 512,
        "height": 512,
        "restore_faces": False,
        "tiling": False,
        #"eta": 0,
        #"s_churn": 0,
        #"s_tmax": 0,
        #"s_tmin": 0,
        #"s_noise": 1,
        #"override_settings": {},
        "sampler_index": "Euler a"
    }

    if not samplers:
        samplers = [
            "Euler a",
            "Euler",
            "LMS",
            "Heun",
            "LMS Karras",
            "DPM fast",
            "DPM adaptive",
            "DPM2",
            "DPM2 a",
            "DPM2 Karras",
            "DPM2 a Karras",
            "DPM++ 2S a Karras",
            "DPM++ 2M Karras",
            "DPM++ 2S a",
            "DPM++ 2M",
            "DDIM",
            "PLMS"
        ]
    return payload, samplers

### Model Tester

In [3]:
# Some Prompt you want to use for Model Testing
prompt = "sports car, photorealistic, natural lighting, masterpiece, high quality, hi res, highres"
# Some Negative Prompt you want to use for Model Testing
negative_prompt = "deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality, deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality"
# Seed, if set to -1 autogenerate will be done
seed = -1
# How many Steps you want
steps = 90
# Which samplers you want to check
# If none are supplied, it will default to all
samplers = ["Euler a",
            "DPM++ 2S a Karras",
            "DPM++ 2M Karras",
            "DPM++ 2S a",
            "DPM++ 2M", ]
# Names of the models you want to check in an array
models = ["v1-5-pruned.ckpt [a9263745]",
          "sd-v1-4-full-ema.ckpt [06c50424]",
          "v2-1_768-ema-pruned.ckpt [4bdfc29c]"]
# Get Payload and Sampler information from initialise function
# You can also just do that by hand
payload, samplers = initialise(seed, prompt, negative_prompt, steps, samplers)

# Generate Images for each model and sampler
for model in models:
    generate_images(url, samplers, payload, model)

Change to model v1-5-pruned.ckpt [a9263745]


NameError: name 'url' is not defined

### Artist Tester

In [ ]:
# Array of artist you want to test against
artists = ["Michael & Inessa Garmash",
           "Ruan Jia",
           "Edmund Dulac",
           "Rumiko Takahashi",
           "Eileen Agar",
           "Katsushika Hokusai",
           "Hokusai",
           "Nicolas Delort",
           "Andy Kehoe",
           "Conrad Roset",
           "Helen Frankenthaler",
           "Giotto Di Bondone",
           "Takashi Murakami",
           "Eduardo Paolozzi",
           "Rebecca Louise Law",
           "Harumi Hironaka",
           "Alexander Calder",
           "Charles Blackman",
           "Jeremiah Ketner",
           "Jim Mahfood",
           "sepulte",
           "pixelsketcher",
           "inubashiri",
           "personalami",
           "chunie",
           "Pino Daeni",
           "Kentaro Miura",
           "Carl Larsson",
           "Bernard Buffet",
           "Don Bluth",
           "Naoko Takeuchi",
           "Ethan Van Sciver",
           "Barry Windsor Smith",
           "Milton Caniff",
           "Osamu Tezuka",
           "Toei Animations", ]

# Sampler to use
# I usually only take 1-2 different samplers
samplers = ["Euler a"]
# The Prompt to use with the artists
prompt = "sports car, photorealistic, natural lighting, masterpiece, high quality, hi res, highres"
# The Negative Prompt to use with the artists
negative_prompt = "deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality, deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality"
# Seed, if -1 it will be random
seed = 4805326764
# How many steps
steps = 20
# Which model to use, if None, it will just use whatever is loaded
model = "v2-1_768-ema-pruned.ckpt [4bdfc29c]"

# Get Payload and Sampler information from initialise function
# You can also just do that by hand
payload, samplers = initialise(seed, prompt, negative_prompt, steps, samplers)

generate_images(url, samplers, payload, model=model, artists=artists)

### Prompt Tester

In [ ]:
# Prompt checker
# Prompts to use for the prompt checker
# This string will be split by ", " and these prompts will be evaluated
prompt = "photorealistic, natural lighting, masterpiece, high quality, hi res, highres"
# Negative Prompts to add
negative_prompt = "deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality, deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality"
# How many steps you want to take
steps = 60
# Seed, if -1 it will be random
seed = 4205326764
# Sampler to use
# I usually only take 1-2 different samplers
samplers = ["Euler a"]
# Which model to use, if None, it will just use whatever is loaded
model = "v2-1_768-ema-pruned.ckpt [4bdfc29c]"
# Static Keyword which will be tested with every split part of the prompt
keyword_test="sports car"
# Get Payload and Sampler information from initialise function
# You can also just do that by hand
payload, samplers = initialise(seed, prompt, negative_prompt, steps, samplers)

generate_images(url, samplers, payload, model=model, artists=None, keyword_test=keyword_test)